In [276]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [277]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'

In [278]:
img_width, img_height = 30, 120

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [279]:
train_datagen = ImageDataGenerator(
    rescale= 1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(
    rescale=1. / 255)

In [280]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 48 images belonging to 2 classes.
Found 15 images belonging to 2 classes.
Found 5 images belonging to 1 classes.


In [281]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [282]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 28, 118, 32)       896       
_________________________________________________________________
activation_109 (Activation)  (None, 28, 118, 32)       0         
_________________________________________________________________
max_pooling2d_65 (MaxPooling (None, 14, 59, 32)        0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 12, 57, 32)        9248      
_________________________________________________________________
activation_110 (Activation)  (None, 12, 57, 32)        0         
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 6, 28, 32)         0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 4, 26, 64)         18496     
__________

In [285]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [286]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
48/48 [==============================] - 2s 31ms/step - loss: 0.8278 - acc: 0.5208 - val_loss: 0.8773 - val_acc: 0.5333
Epoch 2/10
48/48 [==============================] - 0s 10ms/step - loss: 0.7254 - acc: 0.6667 - val_loss: 0.6891 - val_acc: 0.5333
Epoch 3/10
48/48 [==============================] - 0s 10ms/step - loss: 0.7209 - acc: 0.6667 - val_loss: 0.6891 - val_acc: 0.5333
Epoch 4/10
48/48 [==============================] - 0s 10ms/step - loss: 0.7471 - acc: 0.6667 - val_loss: 0.6932 - val_acc: 0.5333
Epoch 5/10
48/48 [==============================] - 0s 10ms/step - loss: 0.6910 - acc: 0.6667 - val_loss: 0.6768 - val_acc: 0.5333
Epoch 6/10
48/48 [==============================] - 0s 9ms/step - loss: 0.6893 - acc: 0.6667 - val_loss: 0.7969 - val_acc: 0.5333
Epoch 7/10
48/48 [==============================] - 0s 10ms/step - loss: 0.5992 - acc: 0.6667 - val_loss: 0.7733 - val_acc: 0.5333
Epoch 8/10
48/48 [==============================] - 0s 10ms/step - loss: 0.4594 - ac

In [292]:
model.evaluate_generator(generator=validation_generator, steps = 15)

[0.2650060756248422, 0.9333333333333333]

In [293]:
test_generator.reset()
pred=model.predict_generator(test_generator, steps = 5)

In [294]:
import numpy as np
predicted_class_indices=np.argmax(pred,axis=1)

In [295]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [296]:
import pandas as pd
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)